In [1]:
import cv2
import mediapipe as mp
import time
import math

In [6]:
mp.solutions.pose.Pose??

In [7]:
class Posture():
    def __init__(self,staticImgMode=False,modelComp=1,smoothLms=True,enableSegment=False,
                                smoothSegment=True,minDetectConf=0.5,minTrackingConf=0.5):
        self.staticImgMode=False
        self.modelComp=1
        self.smoothLms=True
        self.enablSegment=False
        self.smoothSegment=True
        self.minDetectConf=0.5
        self.minTrackingConf=0.5
        self.mpPose=mp.solutions.pose
        self.pose=self.mpPose.Pose()
        self.mpDraw=mp.solutions.drawing_utils
    
    def findPosture(self,img,draw=True):
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)      
        self.results=self.pose.process(imgRGB)
        lmList=[]
        if self.results.pose_landmarks:
            for Id,lm in enumerate(self.results.pose_landmarks.landmark):
                h,w,c=img.shape
                x,y=int(lm.x*w),int(lm.y*h)
                #print(x,y)
                lmList.append([Id,x,y])
            if draw:
                self.mpDraw.draw_landmarks(img,self.results.pose_landmarks,self.mpPose.POSE_CONNECTIONS)
        
        return lmList
    
        
    def findAngle(self,img,a,b,c,draw=True):
        x1,y1=a[1:]
        x2,y2=b[1:]
        x3,y3=c[1:]
        angle=math.degrees(math.atan2(y3-y2,x3-x2)-math.atan2(y1-y2,x1-x2))
        if angle<0:
            angle+=360
        if draw:
            cv2.circle(img,(x1,y1),7,(0,0,255),-1)
            cv2.circle(img,(x1,y1),12,(0,0,255),1)
            
            cv2.circle(img,(x2,y2),7,(0,0,255),-1)
            cv2.circle(img,(x2,y2),11,(0,0,255),1)
            
            cv2.circle(img,(x3,y3),7,(0,0,255),-1)
            cv2.circle(img,(x3,y3),11,(0,0,255),1)
            
            cv2.line(img,(x1,y1),(x2,y2),(84, 248, 136),3)
            cv2.line(img,(x2,y2),(x3,y3),(84, 248, 136),3)
        return angle
    
    def findCount(self,img,per,flag=0,count=0):
        if per==100 and flag==0:
            flag=1
        elif per==0 and flag==1:
            flag=0
            count+=1
            
        return flag,count

In [3]:
def main():
    cap=cv2.VideoCapture('weights.mp4')
    pTime=0
    cTime=0
    p=Posture()
    while cap.isOpened():
        ret,frame=cap.read()
        img=cv2.resize(frame,(640,480))
        cTime=time.time()
        fps=1/(cTime-pTime)
        pTime=cTime
        p.findPosture(img)
        cv2.putText(img,str(int(fps)),(50,250),cv2.FONT_HERSHEY_PLAIN,5,(0,0,255),2)
        cv2.imshow('Posture',img)
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [14]:
if __name__=='__main__':
    main()